## LCEL 인터페이스


사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain 을 생성합니다.

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

## stream: 실시간 출력


이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [6]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 여러 가지 다양한 형태의 자극을 동시에 활용하여 정보를 전달하거나 의사소통하는 방식을 말합니다. 이러한 다양한 형태의 자극은 텍스트, 이미지, 음성, 동영상 등으로 구성될 수 있습니다. 멀티모달은 사용자에게 더욱 풍부한 경험을 제공하고 정보를 보다 효과적으로 전달할 수 있는 장점을 가지고 있습니다.

## invoke: 호출


`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [7]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 대화형 인공지능 모델로, 다양한 주제에 대해 자연스럽고 유창한 대화가 가능합니다. 사용자의 질문에 효과적으로 응답하고 맞춤형 정보를 제공하여 상호작용을 즐길 수 있습니다. 자연어 처리 기술을 기반으로하며, 챗봇, 가이드, 문제 해결 및 다른 응용 분야에 활용됩니다.'

## batch: 배치(단위 실행)


함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.

In [8]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 자연어 처리 기술을 기반으로 한 대화형 인공지능 챗봇이다. 이 기술은 대화를 통해 사용자에게 정보를 제공하고 질문에 답변할 수 있다. 또한 더 복잡한 대화와 상황에 맞는 응답을 생성할 수 있다.',
 'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼이다. 사용자들은 다양한 필터와 효과를 이용하여 창의적인 콘텐츠를 작성하며 팔로워들과 소통한다. 인기 있는 인플루언서들과 브랜드들도 활발하게 활동하고 있어 전 세계적으로 매력적인 플랫폼으로 자리매김하고 있다.']

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

`config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [9]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 자연어 처리 모델로, 사용자와의 대화를 통해 다양한 주제에 대해 대화하고 질문에 답변을 제공합니다. 사용자의 질문이나 입력에 따라 해당 주제에 대한 지식을 활용하여 응답을 생성하며, 챗봇이 자연스럽게 대화를 이어나갈 수 있도록 합니다. ChatGPT는 딥 러닝을 기반으로 작동하며, 뛰어난 자연어 이해 능력을 통해 사용자와의 대화를 자연스럽고 유익하게 이끌어냅니다.',
 'Instagram은 SNS 애플리케이션으로, 사진과 동영상을 공유하고 다른 사용자들과 소통할 수 있는 플랫폼이다. 사용자들은 해시태그를 통해 관심사에 맞는 콘텐츠를 찾고 팔로우하는 다양한 계정을 구독할 수 있다. 인기 있는 인플루언서와 브랜드들도 활발하게 활동하며, 시각적으로 매력적이고 다양한 콘텐츠를 제공한다.',
 '멀티모달은 두 가지 이상의 다양한 형식의 정보나 자료를 같이 사용하여 전달하는 방식이다.\n\n이는 시각, 청각, 촉각, 말 등 다양한 감각을 활용하여 다양한 형식의 정보 전달을 가능하게 한다.\n\n멀티모달을 활용하면 정보를 보다 효과적으로 전달할 수 있을 뿐만 아니라 수용자들의 이해도와 기억률이 더 높아진다.',
 '프로그래밍은 컴퓨터에게 작업을 시키기 위해 코드를 작성하는 과정이다. 코드는 컴퓨터가 이해할 수 있는 명령어로 이루어져 있고, 프로그래머가 원하는 작업을 수행하도록 정확하고 명확하게 작성되어야 한다. 프로그래밍을 잘하려면 컴퓨터 과학에 대한 이해와 논리적 사고력이 필요하다.',
 '머신러닝은 컴퓨터 시스템이 데이터에서 학습하고 패턴을 발견하여 자동적으로 작업을 수행하는 인공지능의 한 분야입니다. 이를 통해 머신러닝은 예측, 분류, 클러스터링과 같은 다양한 작업을 수행하며 인간의 개입 없이 스스로 학습하고 발전할 수 있습니다. 머신러닝을 통해 문제를 해결하거나 창의적인 목표를 달성하기 위한 새로운 방법을 발견할 수 있습니다.']

## async stream: 비동기 스트림


함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [10]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 전 세계에서 가장 인기 있는 동영상 공유 플랫폼으로, 사용자들이 영상을 업로드하고 시청할 수 있는 서비스이다. 또한 다양한 콘텐츠를 제공하며, 유튜버들은 시청자들과 소통하고 수익을 창출할 수 있다. 인터넷 환경에서 가장 많은 시청자들을 보유하고 있으며, 광고 및 스폰서십을 통해 수익을 창출할 수 있다.

## async invoke: 비동기 호출


`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.


In [11]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [12]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

'NVDA는 "NVIDIA Corporation"의 약자로, 전 세계적으로 유명한 그래픽 처리 유닛 및 시각 컴퓨팅 기술을 개발하는 기업이다. NVDA의 제품은 주로 게임 산업과 인공지능 및 자율 주행차량 분야에서 사용되며, 세계적으로 높은 평판을 지니고 있다. 또한 NVDA 주식은 전 세계적인 기술 기업 중에서 높은 시가 총액을 가지고 있는 주식 중 하나이다.'

## async batch: 비동기 배치


함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있습니다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [13]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [14]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['YouTube는 비디오 공유 플랫폼으로, 사용자들은 자신이 만든 동영상을 업로드하고 공유할 수 있다. 또한 다른 사용자들이 업로드한 동영상을 시청하고 댓글을 남기거나 구독할 수 있다. 시청자들은 그들이 관심 있는 주제나 콘텐츠를 쉽게 찾아 시청할 수 있다.',
 'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼이며, 사용자들이 팔로워들과 소통하고 콘텐츠를 공유하는 데에 주로 사용됩니다. 해시태그를 통해 관심사에 맞는 게시물을 발견하고 탐색할 수도 있습니다.',
 'Facebook은 전 세계적으로 가장 대중적인 소셜 네트워크 서비스 중 하나로, 사용자들이 친구들과 소통하고 커뮤니케이션할 수 있는 플랫폼이다. 사진, 동영상, 글 등 다양한 콘텐츠를 공유하고 이용자들 간에 소셜 그래프를 형성하여 다양한 활동을 할 수 있다. 또한 광고 및 비즈니스 활동을 위한 플랫폼으로도 널리 이용되고 있다.']

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행합니다.


`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.


In [15]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메서드를 호출합니다.

이때, `country`이라는 키에 `대한민국`라는 값을 가진 딕셔너리를 인자로 전달합니다.


In [16]:
# chain1 를 실행합니다.
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울이다.'

이번에는 `chain2.invoke()` 를 호출합니다. `country` 키에 다른 국가인 `미국` 을 전달합니다.


In [17]:
# chain2 를 실행합니다.
chain2.invoke({"country": "미국"})

'미국의 면적은 약 9,833,520 km² 입니다.'

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행합니다.

이 예제에서는 `대한민국`라는 주제를 `invoke` 메서드에 전달하여 실행합니다.


In [18]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '서울이에요.', 'area': '대한민국의 면적은 약 100,363 제곱 킬로미터 입니다.'}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.


`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.


In [19]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울이다.', '미국의 수도는 워싱턴 D.C.입니다.']

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(batch)를 수행합니다.

이 예시에서는 `대한민국`와 `미국`라는 두 가지 국가에 대한 처리를 요청합니다.


In [20]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 총 면적은 약 100,363.34km² 입니다.', '미국의 면적은 약 9,833,517 제곱 킬로미터입니다.']

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`와 `미국` 두 나라에 대한 데이터를 배치 처리합니다.


In [21]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울이야.', 'area': '대한민국의 총 면적은 약 100,363 제곱 킬로미터 입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.이다.', 'area': '미국의 면적은 약 9,833,520km2 입니다.'}]